In [11]:
import polars as pl
import requests
from dotenv import load_dotenv
import os
import urllib.parse

load_dotenv()

def get_nzb_results(query):
    api_key = os.getenv("DRUNKEN_SLUG_API_KEY")
    encoded_query = urllib.parse.quote(query)
    url = f"https://drunkenslug.com/api?t=search&o=json&apikey={api_key}&q={encoded_query}"
    response = requests.get(url)
    response_json = response.json()
    return response_json


In [ ]:
nzb_results = get_nzb_results('allgirlmassage "20 08 24"')
nzb_results["item"]
nzb_results_df = pl.DataFrame(nzb_results["item"])

# Extract category and size from newznab:attr
nzb_results_df = nzb_results_df.with_columns(
    [
        pl.col("newznab:attr").list.get(0).struct.field("_value").alias("category"),
        pl.col("newznab:attr")
        .list.get(1)
        .struct.field("_value")
        .cast(pl.Int64)
        .alias("size"),
        pl.col("guid").struct.field("text").alias("item_url"),
    ]
)
nzb_results_df = nzb_results_df.sort(by="size", descending=True)
nzb_results_df.select(
    pl.col("title"), pl.col("size"), pl.col("link"), pl.col("category"), pl.col("item_url")
)

In [ ]:
nzb_results_df.slice(1, 1).select(pl.col("link")).to_series().to_list()